In [18]:
import pandas as pd
import matplotlib as plt
import numpy as np

# Loading Dataset


In [19]:
v1_df = pd.read_parquet("hf://datasets/kinisi/gym_kr2-PickYCB-v1_generated/data/chunk-000/episode_000000.parquet")

In [20]:
v1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   observation.images.head_camera  57 non-null     object 
 1   action                          57 non-null     object 
 2   observation.state               57 non-null     object 
 3   timestamp                       57 non-null     float32
 4   frame_index                     57 non-null     int64  
 5   episode_index                   57 non-null     int64  
 6   index                           57 non-null     int64  
 7   task_index                      57 non-null     int64  
dtypes: float32(1), int64(4), object(3)
memory usage: 3.5+ KB


In [21]:
v1_df.head()

,observation.images.head_camera,action,observation.state,timestamp,frame_index,episode_index,index,task_index
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[0.0, 0.0, 0.00096, -0.00096, -0.6, -0.3791001...","[0.018257216, 0.01825725, 0.00011348083, -0.00...",0.000000,0,0,0,0
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[0.0, 0.0, 0.018409671, -0.018409671, -0.6, -0...","[0.0022636736, 0.0022581709, 0.017093448, -0.0...",0.033333,1,0,1,0
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[0.0, 0.0, 0.038400035, -0.038400035, -0.6, -0...","[0.0022636736, 0.0022581709, 0.037060156, -0.0...",0.066667,2,0,2,0
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[0.0, 0.0, 0.0584, -0.0584, -0.6, -0.361041, 0...","[0.0022636736, 0.0022581709, 0.05701695, -0.05...",0.100000,3,0,3,0
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[0.0, 0.0, 0.0784, -0.0784, -0.6, -0.26104364,...","[0.0022636736, 0.0022581709, 0.07722339, -0.07...",0.133333,4,0,4,0


In [22]:
v1_df['observation.state'][0]

array([ 1.8257216e-02,  1.8257249e-02,  1.1348083e-04, -2.8994158e-04,
       -5.9996909e-01, -3.7563673e-01,  4.6909261e-01, -1.7981676e+00,
        1.7981454e+00,  1.8893399e-03, -1.8990268e-03, -1.3425452e+00,
        1.2449249e+00,  1.0007094e+00, -8.9245963e-01,  2.0212786e+00,
       -2.0104916e+00,  3.0897100e+00, -3.0916495e+00,  4.1746971e-06,
       -8.0988502e-06,  2.5658667e-06, -1.4874968e-06, -1.8485051e-06,
       -9.3905965e-06,  3.5244777e-06, -4.2000584e-10,  6.2228332e-07,
        2.4975833e-07,  1.5961911e-06, -1.4405556e-06], dtype=float32)

# Creating New Dataframe from 'observation.state' column

In [23]:
state_df = pd.DataFrame(v1_df['observation.state'].tolist())
state_df.columns = [f'state_{i}' for i in range(state_df.shape[1])]

In [24]:
state_df.head()

,state_0,state_1,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state_9,...,state_21,state_22,state_23,state_24,state_25,state_26,state_27,state_28,state_29,state_30
0,0.018257,0.018257,0.000113,-0.000290,-0.599969,-0.375637,0.469093,-1.798168,1.798145,0.001889,...,0.000003,-0.000001,-1.848505e-06,-9.390596e-06,3.524478e-06,-4.200058e-10,6.222833e-07,2.497583e-07,1.596191e-06,-0.000001
1,0.002264,0.002258,0.017093,-0.017100,-0.599965,-0.460399,0.389063,-1.714065,1.775322,0.085883,...,0.003857,-0.003854,-1.672838e-06,4.411417e-07,8.706263e-07,-6.068399e-11,-3.849686e-03,3.855643e-03,-3.881168e-06,-0.000002
2,0.002264,0.002258,0.037060,-0.037099,-0.599973,-0.465861,0.311864,-1.614136,1.766498,0.185800,...,0.042323,-0.042319,-9.106772e-07,1.812524e-07,6.959257e-07,-7.182561e-11,-4.231729e-02,4.231977e-02,-2.184701e-06,-0.000001
3,0.002264,0.002258,0.057017,-0.057066,-0.599959,-0.367004,0.225062,-1.514135,1.764801,0.225142,...,0.165622,-0.165615,-1.330728e-06,2.455770e-07,8.137070e-07,-8.340395e-11,-1.656132e-01,1.656151e-01,-3.283876e-06,-0.000002
4,0.002264,0.002258,0.077223,-0.077178,-0.600005,-0.267284,0.136047,-1.414119,1.769840,0.128179,...,0.380568,-0.380563,-1.435122e-06,-3.511445e-06,1.927374e-06,-2.018715e-10,-3.805638e-01,3.805639e-01,-7.682806e-08,-0.000003


In [25]:
state_v1 = pd.concat([v1_df[['timestamp', 'task_index', 'episode_index']],state_df],axis=1)

In [26]:
state_v1.head()

,timestamp,task_index,episode_index,state_0,state_1,state_2,state_3,state_4,state_5,state_6,...,state_21,state_22,state_23,state_24,state_25,state_26,state_27,state_28,state_29,state_30
0,0.000000,0,0,0.018257,0.018257,0.000113,-0.000290,-0.599969,-0.375637,0.469093,...,0.000003,-0.000001,-1.848505e-06,-9.390596e-06,3.524478e-06,-4.200058e-10,6.222833e-07,2.497583e-07,1.596191e-06,-0.000001
1,0.033333,0,0,0.002264,0.002258,0.017093,-0.017100,-0.599965,-0.460399,0.389063,...,0.003857,-0.003854,-1.672838e-06,4.411417e-07,8.706263e-07,-6.068399e-11,-3.849686e-03,3.855643e-03,-3.881168e-06,-0.000002
2,0.066667,0,0,0.002264,0.002258,0.037060,-0.037099,-0.599973,-0.465861,0.311864,...,0.042323,-0.042319,-9.106772e-07,1.812524e-07,6.959257e-07,-7.182561e-11,-4.231729e-02,4.231977e-02,-2.184701e-06,-0.000001
3,0.100000,0,0,0.002264,0.002258,0.057017,-0.057066,-0.599959,-0.367004,0.225062,...,0.165622,-0.165615,-1.330728e-06,2.455770e-07,8.137070e-07,-8.340395e-11,-1.656132e-01,1.656151e-01,-3.283876e-06,-0.000002
4,0.133333,0,0,0.002264,0.002258,0.077223,-0.077178,-0.600005,-0.267284,0.136047,...,0.380568,-0.380563,-1.435122e-06,-3.511445e-06,1.927374e-06,-2.018715e-10,-3.805638e-01,3.805639e-01,-7.682806e-08,-0.000003
